In [8]:
#Question 1: What is the most popular attraction to visit in the park?
import sqlite3
from datetime import timedelta
import operator

db_filename = 'dinofunworld.db'
conn = sqlite3. connect(db_filename) 
cursor = conn.cursor()

cursor.execute("SELECT name FROM checkin, attraction WHERE checkin.attraction = attraction.attractionid GROUP BY name ORDER BY count(*) DESC limit 1")
print(cursor.fetchone()[0])

Atmosfear


In [10]:
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
cursor.execute("SELECT attractionid from attraction WHERE LOWER(category) like LOWER('%ride%')")

duration_dict = {}
for i in cursor.fetchall():
    ride_id = i[0]
    cursor.execute("SELECT duration FROM checkin WHERE attraction=" + str(ride_id))
    
    ride_duration = 0    
    for j in cursor.fetchall():
        count = 0
        try:
            ride_duration += timedelta(int(j[0].split(':')[0]), int(j[0].split(':')[1]), int(j[0].split(':')[2])).total_seconds()
            count += 1
        except (ValueError, AttributeError) as e:
            pass
    duration_dict[ride_id] = ride_duration

attraction_id = max(duration_dict.items(), key=operator.itemgetter(1))[0]
cursor.execute("SELECT Name FROM attraction WHERE AttractionID=" + str(attraction_id))
print(cursor.fetchone()[0])

Flight of the Swingodon


In [11]:
# Question 3: Which Fast Food offering in the park has the fewest visitors?
cursor.execute("SELECT name FROM attraction, checkin WHERE attraction.attractionid = checkin.attraction and attraction.type='Fast Food' GROUP BY name order by count(*) asc limit 1")
print(cursor.fetchone()[0])

Theresaur Food Stop


In [12]:
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline. 

cursor.execute("SELECT attractionid from attraction WHERE LOWER(category) like LOWER('%ride%')")

ride_dict = {}
for i in cursor.fetchall():
    ride_id = i[0]
    cursor.execute("SELECT duration FROM checkin WHERE attraction=" + str(ride_id))
    
    ride_duration = 0    
    for j in cursor.fetchall():
        count = 0
        try:
            ride_duration += timedelta(int(j[0].split(':')[0]), int(j[0].split(':')[1]), int(j[0].split(':')[2])).total_seconds()
            count += 1
        except (ValueError, AttributeError) as e:
            pass
    ride_dict[ride_id] = [ride_duration]
    
    cursor.execute("SELECT count(*) FROM checkin WHERE attraction=" + str(ride_id))
    checkin_count = cursor.fetchone()[0]
    ride_dict[ride_id].append(checkin_count)
    
visit_ride = list(ride_dict.keys())[0]
wait_ride = visit_ride
both_ride = visit_ride
for ride in ride_dict:
    if ride_dict[ride][0] < ride_dict[wait_ride][0]:
        wait_ride = ride
    if ride_dict[ride][1] > ride_dict[visit_ride][1]:
        visit_ride = ride
    if (ride_dict[ride][0] < ride_dict[both_ride][0]) and (ride_dict[ride][1] > ride_dict[both_ride][1]):
        both_ride = ride
        
cursor.execute("SELECT name FROM attraction WHERE attractionid in (" + str(visit_ride) + "," + str(wait_ride) + "," + str(both_ride) + ")")
res = cursor.fetchall()
print([item[0] for item in res])

['Wrightiraptor Mountain', 'Atmosfear', 'Dykesadactyl Thrill']
